### Theoretical summary

Denote the ground truth box as $G$, the predicted/proposed region as $P$ with cordinates:

$$G = [G_x,G_y,G_w,G_h],\ P = [P_x,P_y,P_w,P_h]$$

The linear model expression is:

$$t_*(P) = w_*^T\phi (P) + \epsilon$$

Weights are constructed by ridge regression, thus:
$$\hat{w}_* = \arg \min \{ ||t_*-w_*^T\phi(P)||^2 +\lambda ||w_*||^2 \}$$

Here, $\phi(P)$ is features extracted by the AlexNet model trained before, and $t_*$ verifies with different subscript like:
$$t_x = \frac{G_x-P_x}{P_w}$$
$$t_y = \frac{G_y-P_y}{P_wh}$$
$$t_w = \log(\frac{G_w}{P_w})$$
$$t_h = \log(\frac{G_h}{P_h})$$

Hence, predicted ground truth boxes can be generated by the reverse versions of expresstions above. 

### Load data

In [1]:
import numpy as np
import pandas as pd

T = pd.read_csv('Data/BBR/T.csv').drop('Unnamed: 0',axis=1)
phi = np.load('Data/BBR/phi.npy')

### Ridge regression

In [2]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def ridge(alphas,str_ind):
    
    X,y = phi,np.array(T[str_ind]).reshape([len(T),1])
    X_train,  X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)
    
    kfold = KFold(n_splits=5)
    ridge1 = RidgeCV(alphas = alphas,fit_intercept=False,scoring='r2').fit(X_train,y_train)
    alpha = ridge1.alpha_
    
    print('best alpha is:',alpha)
    
    ridge = Ridge(alpha=alpha)
    scores = cross_val_score(ridge,X_train,y_train,
                             cv=5,scoring='r2')
    print('5-fold r2 is:%.2f ' % (scores.mean()))
    
    ridge.fit(X_train,y_train)
    print('r2 score on train set: %.2f' % (ridge.score(X_train,y_train)))
    print('MSE on train set: %.6f' % (mean_squared_error(y_train, ridge.predict(X_train))))
    print('r2 score on test set: %.2f' % (ridge.score(X_test,y_test)))
    print('MSE on test set: %.6f' % (mean_squared_error(y_test, ridge.predict(X_test))))
                                      
    ridge.fit(X,y)
    
    return ridge

In [3]:
reg_tx = ridge(alphas = np.linspace(start=5, stop=15, num=10),
               str_ind='t_x')

best alpha is: 5.0
5-fold r2 is:0.12 
r2 score on train set: 0.22
MSE on train set: 0.000426
r2 score on test set: 0.17
MSE on test set: 0.000459


In [4]:
reg_ty = ridge(alphas = np.linspace(start=20, stop=30, num=10),
               str_ind='t_y')

best alpha is: 20.0
5-fold r2 is:0.13 
r2 score on train set: 0.19
MSE on train set: 0.000419
r2 score on test set: 0.14
MSE on test set: 0.000431


In [5]:
reg_tw = ridge(alphas = np.linspace(start=5, stop=15, num=10),
               str_ind='t_w')

best alpha is: 5.0
5-fold r2 is:0.15 
r2 score on train set: 0.24
MSE on train set: 0.002079
r2 score on test set: 0.17
MSE on test set: 0.002215


In [6]:
reg_th = ridge(alphas = np.linspace(start=10, stop=20, num=10),
               str_ind='t_h')

best alpha is: 10.0
5-fold r2 is:0.17 
r2 score on train set: 0.24
MSE on train set: 0.001794
r2 score on test set: 0.18
MSE on test set: 0.001988


### Save model

In [7]:
import pickle

pickle.dump(reg_tx, open('Model/reg_tx.sav', 'wb'))
pickle.dump(reg_ty, open('Model/reg_ty.sav', 'wb'))
pickle.dump(reg_tw, open('Model/reg_tw.sav', 'wb'))
pickle.dump(reg_th, open('Model/reg_th.sav', 'wb'))